In [26]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [27]:
import pandas as pd
import numpy as np

In [28]:
pd.set_option('display.max_columns', None)

# 2. 파이썬을 통한 전처리
- 원본에서 밀려적힌것을 수작업으로 수정한 파일

In [29]:
meta2017 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2017.csv',encoding = 'utf-8')
meta2018 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2018.csv',encoding = 'utf-8')
meta2019 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2019.csv',encoding = 'utf-8')
meta2020 = pd.read_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/1차 전처리 : 수작업/KNOW_2020.csv',encoding = 'utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (179) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
#knowcode 결측치 제거
meta2017 = meta2017[meta2017['knowcode'] != 9999999]
meta2018 = meta2018[meta2018['knowcode'] != 9999999]
meta2019 = meta2019[meta2019['knowcode'] != 9999999]
meta2020 = meta2020[meta2020['knowcode'] != 9999999]

In [31]:
metas = [meta2017,meta2018,meta2019,meta2020]

## 2-1. 텍스트 데이터 컬럼을 지운다
- 텍스트 관련 컬럼 사용은 추후에 진행

In [32]:
#텍스트 컬럼 지우기
meta2017 = meta2017.drop(['idx', 'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq19_1', 'bq30','bq31', 'bq32', 'bq33', 'bq34', 'bq38_1'], axis = 1)
meta2018 = meta2018.drop(['idx', 'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2','bq28_1', 'bq29', 'bq30', 'bq31', 'bq32', 'bq33','bq37_1', 'bq40'], axis = 1)
meta2019 = meta2019.drop(['idx', 'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2','bq18_10', 'bq20_1', 'bq22', 'bq23', 'bq24', 'bq27_1'], axis = 1)
meta2020 = meta2020.drop(['idx', 'bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq18_10', 'bq18_10', 'bq20_1', 'bq26_1'], axis = 1)
metas = [meta2017,meta2018,meta2019,meta2020]
for meta in metas:
  print(len(meta.columns))

144
128
142
177


텍스트 컬럼 제거한 나머지 컬럼들 확인

컬럼이 많아서 따로 함수를 써서 본다

In [33]:
#info 돌아가면서 확인
def all_col_info(metas): 
  year = 2017
  for meta in metas:
      for i in range(25,len(meta.columns),25):
          print(year)
          meta[meta.columns[i-25:i]].info()
      meta[meta.columns[i:]].info()
      year += 1
      print()

In [34]:
all_col_info(metas)

2017
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9104 entries, 0 to 9485
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   aq1_1   9104 non-null   int64 
 1   aq1_2   9104 non-null   object
 2   aq2_1   9104 non-null   int64 
 3   aq2_2   9104 non-null   object
 4   aq3_1   9104 non-null   int64 
 5   aq3_2   9104 non-null   object
 6   aq4_1   9104 non-null   int64 
 7   aq4_2   9104 non-null   object
 8   aq5_1   9104 non-null   int64 
 9   aq5_2   9104 non-null   object
 10  aq6_1   9104 non-null   int64 
 11  aq6_2   9104 non-null   object
 12  aq7_1   9104 non-null   int64 
 13  aq7_2   9104 non-null   object
 14  aq8_1   9104 non-null   int64 
 15  aq8_2   9104 non-null   object
 16  aq9_1   9104 non-null   int64 
 17  aq9_2   9104 non-null   object
 18  aq10_1  9104 non-null   int64 
 19  aq10_2  9104 non-null   object
 20  aq11_1  9104 non-null   int64 
 21  aq11_2  9104 non-null   object
 22  aq12_1  9104 non-nu

숫자로 되어있는 컬럼들이지만 스트링으로 저장된 것을 볼 수 있다
- 이유1: 설문지 결측치를 ' ' 으로 저장했기때문
- 이유2: 숫자값이 스트링으로 저장되어있기 때문

## 2-2. 결측치 처리 및 Int로 통일

### 결측치 처리

In [35]:
#결측치 처리 ' '으로 입력된걸 진짜 결측인 np.nan으로 바꿔줌
year = 2017
metas = [meta2017,meta2018,meta2019,meta2020]
for meta in metas:
    print(year)
    for col in  list(meta.columns):
        for i in range(len(meta)):
            if meta[col].iloc[i] == ' ':
              meta[col].iloc[i] = np.nan
    year +=1

2017


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


2018
2019
2020


결측치를 0으로 바꿔줌

(결측치를 최빈값으로 바꿔도 줘봤으나 성능떨어짐

In [36]:
#결측치 0으로 바꾸기
meta2017.fillna(0,inplace = True)
meta2018.fillna(0,inplace = True)
meta2019.fillna(0,inplace = True)
meta2020.fillna(0,inplace = True)

모든 값들을 Int 로 바꿔줌

In [37]:
#float 및 string 형태의 컬럼들을 int 형태로 바꿔줌
metas = [meta2017,meta2018,meta2019,meta2020]
year = 2017
for meta in metas:
  for col in meta.columns:
    meta[col] = pd.to_numeric(meta[col], errors = 'coerce') #혹시라도 컬럼에 값들을 숫자로 바꾸는데 숫자형이 아닌게 있으면 강제로 nan으로 바꿔준다
    meta[col].fillna(0,inplace = True)#그 후 nan값을 0으로 바꿔줌
    meta[col] = meta[col].astype(int)
  print(year)
  year +=1

2017
2018
2019
2020


## 2-3. 특정 년도 전처리
- 중복체크에 대한 문항 전처리 (2018년)

중복대답을 요구하는 컬럼

- 설문지에서 22번 문항은 하나이나 중복대답인 질문이므로 3개에 컬럼에 나눠 적혀있음)
- 답이 1,3이면 221 컬럼과 223 컬럼에 각각 1,3이 적혀있고 222 컬럼에 0이 적혀야 함
- 근데 작성자의 착각으로 221 컬럼에 다른 컬럼 답이 적혀있기도 해서 원래 위치로 보내줘야함

In [38]:
# 정답을 원래위치로 보내주는 함수
def to_other_col(col1,col2,df,number):
  part = df[df[col1] == number]
  wronglist = list(part.index)
  df[col1][wronglist] = 0
  df[col2][wronglist] = number

### 2018년도

맨앞 컬럼에 다른 컬럼 답들이 있다

In [39]:
print(meta2018['bq221'].unique())
print(meta2018['bq222'].unique())
print(meta2018['bq223'].unique())

[1 0 2 3]
[0 2]
[0 3]


In [40]:
to_other_col('bq221','bq222',meta2018, 2)
to_other_col('bq221','bq223',meta2018, 3)

함수 적용 후 한 각 컬럼에 맞게 답이 들어있게되었다

In [41]:
print(meta2018['bq221'].unique())
print(meta2018['bq222'].unique())
print(meta2018['bq223'].unique())

[1 0]
[0 2]
[0 3]


마찬가지로 다른 중복질문 컬럼에도 적용

In [42]:
print(meta2018['bq231'].unique())
print(meta2018['bq232'].unique())
print(meta2018['bq233'].unique())
print(meta2018['bq234'].unique())
print(meta2018['bq235'].unique())

[1 0 2 5 4 3]
[0 2 4 3]
[0 3]
[0 4]
[0 5]


In [43]:
to_other_col('bq231','bq232',meta2018, 2)
to_other_col('bq231','bq233',meta2018, 3)
to_other_col('bq231','bq234',meta2018, 4)
to_other_col('bq231','bq235',meta2018, 5)
to_other_col('bq232','bq233',meta2018, 3)
to_other_col('bq232','bq234',meta2018, 4)

In [44]:
print(meta2018['bq231'].unique())
print(meta2018['bq232'].unique())
print(meta2018['bq233'].unique())
print(meta2018['bq234'].unique())
print(meta2018['bq235'].unique())

[1 0]
[0 2]
[0 3]
[0 4]
[0 5]


그 다음 컬럼

In [45]:
print(meta2018['bq241'].unique())
print(meta2018['bq242'].unique())
print(meta2018['bq243'].unique())
print(meta2018['bq244'].unique())
print(meta2018['bq245'].unique())

[1 0 4 2 5]
[0 2 3]
[0 3]
[0 4]
[0 5]


In [46]:
to_other_col('bq241','bq242',meta2018, 2)
to_other_col('bq241','bq244',meta2018, 4)
to_other_col('bq241','bq245',meta2018, 5)
to_other_col('bq242','bq243',meta2018, 3)

In [47]:
print(meta2018['bq241'].unique())
print(meta2018['bq242'].unique())
print(meta2018['bq243'].unique())
print(meta2018['bq244'].unique())
print(meta2018['bq245'].unique())

[1 0]
[0 2]
[0 3]
[0 4]
[0 5]


## 2-4. 특정 컬럼 One Hot Encoding
- 개인의 주관적인 답변이 들어간 값들을 제외한 것들에 대해 One Hot Encoding 진행
- 답이 2개인것은 컬럼 두개로 나눈다
- 카테고리로 분류할 수 있는 질문 (예 - 학력 수준 등등) 은 답변 갯수만큼 컬럼 생성
- 나누는 기준은 우리팀의 주관대로하여 엑셀에 저장해둠
- 아래 불러옴

One Hot Encoding 대상인 컬럼들

In [48]:
#컬럼별로 바꾸려는 타입설정
col_set2017 = pd.read_excel('/gdrive/MyDrive/2조/data/코드정리/전처리 할 것 정리/2017컬럼.xlsx')
col_set2018 = pd.read_excel('/gdrive/MyDrive/2조/data/코드정리/전처리 할 것 정리/2018컬럼.xlsx')
col_set2019 = pd.read_excel('/gdrive/MyDrive/2조/data/코드정리/전처리 할 것 정리/2019컬럼.xlsx')
col_set2020 = pd.read_excel('/gdrive/MyDrive/2조/data/코드정리/전처리 할 것 정리/2020컬럼.xlsx')

In [49]:
col_set2017

,columns,type,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,aq1_1,int,NaN,NaN,NaN,NaN
1,aq1_2,int,NaN,NaN,NaN,NaN
2,aq2_1,int,NaN,NaN,NaN,NaN
3,aq2_2,int,NaN,NaN,NaN,NaN
4,aq3_1,int,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
149,bq39_2,cat,NaN,NaN,NaN,NaN
150,bq40,bi,NaN,NaN,NaN,NaN
151,bq41_1,int,NaN,NaN,NaN,NaN
152,bq41_2,int,NaN,NaN,NaN,NaN


In [50]:
print(col_set2017['type'].unique())
print(col_set2018['type'].unique())
print(col_set2019['type'].unique())
print(col_set2020['type'].unique())

['int' 'cat' 'bi' 'text']
['int' 'cat' 'bi2' 'bi' 'text' 'bi3' 'concat']
['int' 'cat' 'bi' 'text' 'concat']
['int' 'cat' 'bi' 'text']


1. cat: category 별로 one hot encoding -> 학벌수준 등등
2. bi: 1이 Yes이고 2가 No인 질문 또는 성별같은 질문 -> 하나의 컬럼으로 만들어 yes에 해당하는 값을 1로하고 NO에 해당하는것을 0으로
3. bi2: 보기가 3개 이상이지만 binary 형태로 남기고 싶은 질문 -> 1. 없다 2. 적당히 있다 3. 항상 있다 이런 식으로 있을때 2,3을 있다로 묶어주고 1과 3을 떨군다
4. bi3: 중복질문 형태라 이전 전처리 (2-3) 에서 이미 0과 다른값으로 남겨진 컬럼 -> 컬럼 구성이 0,1 또는 0,2 또는 0,3 이런식이라 모두 0,1의 형태로 바꿔줌
5. 'concat' : 보기가 1~7인 질문인데 답변이 1~4인것과 5~7인것이 나눠져있어서 합쳐줘야함
6. 'text' : 'text' 값인 컬럼 -> 이미 사전에 제거했다

In [51]:
def we_get_dummies(df,col_df):
  for idx in range(len(col_df)):
    col_name = col_df['columns'].iloc[idx]
    col_type = col_df['type'].iloc[idx]

    if col_type == 'cat':
      df = pd.get_dummies(df, columns = [col_name],prefix_sep = '__')
      try:
        df = df.drop(f'{col_name}__0',axis = 1)
      except:
        pass
    elif col_type == 'bi':
      df = pd.get_dummies(df, columns = [col_name],prefix_sep = '___')
      df = df.drop(f'{col_name}___2',axis = 1)
      try:
        df = df.drop(f'{col_name}___0',axis = 1)
      except:
        pass
    elif col_type == 'bi2':
      df = pd.get_dummies(df, columns = [col_name],prefix_sep = '___')
      try:
        df[f'{col_name}___1'] = df[f'{col_name}___1'] + df[f'{col_name}___2']
        df = df.drop(f'{col_name}___2',axis = 1)
      except:
        df = df.drop(f'{col_name}___1',axis = 1)
      try:
        df = df.drop(f'{col_name}___0',axis = 1)
      except:
        pass
    elif col_type == 'bi3':
      for i in range(len(df)):
        if df[col_name].iloc[i] != 0:
          df[col_name].iloc[i] = 1
    
  return df

In [52]:
meta2017 = we_get_dummies(meta2017,col_set2017)
meta2018 = we_get_dummies(meta2018,col_set2018)
meta2019 = we_get_dummies(meta2019,col_set2019)
meta2020 = we_get_dummies(meta2020,col_set2020)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [53]:
def column_merge(df, col_df):
  concat_list = []
  for idx in range(len(col_df)):
    col_name = col_df['columns'].iloc[idx]
    col_type = col_df['type'].iloc[idx]

    if col_type == 'concat':
      concat_list.append(col_df['columns'].iloc[idx])


  df[f'{concat_list[0][:4]}_cat'] = df[concat_list[0]] + df[concat_list[1]]
  df = df.drop(concat_list[0], axis = 1)
  df = df.drop(concat_list[1], axis = 1)

  return df

In [54]:
#2018, 2019년에서 1234 와 567 질문이 한컬럼에 있어도 되는데 나뉘어져 있어서 하나로 합치는 작업을 함
meta2018 = column_merge(meta2018,col_set2018)
meta2019 = column_merge(meta2019,col_set2019)

In [55]:
#테스트데이터에서 bq1 질문에 21번을 표기한 답변이 없으므로 제거해버린다
meta2018 = meta2018.drop(['bq1__21'], axis = 1)

## 2-5. 저장

In [56]:
#마지막으로 결측치 확인
all_col_info(metas)

2017
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9104 entries, 0 to 9485
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   aq1_1   9104 non-null   int64
 1   aq1_2   9104 non-null   int64
 2   aq2_1   9104 non-null   int64
 3   aq2_2   9104 non-null   int64
 4   aq3_1   9104 non-null   int64
 5   aq3_2   9104 non-null   int64
 6   aq4_1   9104 non-null   int64
 7   aq4_2   9104 non-null   int64
 8   aq5_1   9104 non-null   int64
 9   aq5_2   9104 non-null   int64
 10  aq6_1   9104 non-null   int64
 11  aq6_2   9104 non-null   int64
 12  aq7_1   9104 non-null   int64
 13  aq7_2   9104 non-null   int64
 14  aq8_1   9104 non-null   int64
 15  aq8_2   9104 non-null   int64
 16  aq9_1   9104 non-null   int64
 17  aq9_2   9104 non-null   int64
 18  aq10_1  9104 non-null   int64
 19  aq10_2  9104 non-null   int64
 20  aq11_1  9104 non-null   int64
 21  aq11_2  9104 non-null   int64
 22  aq12_1  9104 non-null   int64
 23  aq12_2  

In [57]:
meta2017.to_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/2차 전처리 : 파이썬 작업/KNOW_2017_pre.csv',index = False)
meta2018.to_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/2차 전처리 : 파이썬 작업/KNOW_2018_pre.csv',index = False)
meta2019.to_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/2차 전처리 : 파이썬 작업/KNOW_2019_pre.csv',index = False)
meta2020.to_csv('/gdrive/MyDrive/2조/data/코드정리/Data_preprocessed/2차 전처리 : 파이썬 작업/KNOW_2020_pre.csv',index = False)